In [1]:
import sys, re
import pandas as pd
import pyomo.environ as pe
import math, json
from collections import defaultdict

from src.optimization.treatment.preprocess_classes.useful_sets import UsefulSets
from src.optimization.treatment.preprocess_classes.processed_data import ProcessedData

from src.commons.s3_manager import S3Manager

import sys, os, re, datetime, logging
import pandas as pd
from src.commons.s3_manager import S3Manager
from src.optimization.treatment.treatment import treatment_model, read_parameters, process_treatment_results

from src.commons.system_util import SystemUtilities
from src.optimization.treatment.preprocess_data import preprocess_data
from src.optimization.treatment.make_model import make_model
from src.optimization.run_optimization import save_optimization_results
from src.optimization.treatment.generate_output import generate_output
from src.commons.process_results import ProcessResults

logging.getLogger('pyomo.core').setLevel(logging.ERROR)

import src.recommendations.water_recommendations as wr


In [7]:
s3_data, param_file = False, "parameters_permian_scenario5_dav.json"  #please maintain this values once you finish your development
date = None if re.match('linux.*', sys.platform) else datetime.datetime.now()
test, parameters, simulated_period = read_parameters("water", s3_data, param_file, date)

print("    Preprocessing data")
data, useful_sets, attributes_with_time = preprocess_data(parameters, simulated_period,s3_data)
print("    optimizing model...")     #putting data into attributes

    reading parameters file parameters_permian_scenario5_dav.json...
    processing for Permian asset...
    checking directories...
    reading parameters...
    Preprocessing data
    reading data from one drive...
    processing loaded data...
    creating class attributes...
    optimizing model...


In [8]:
TOTAL_EVAP=0
for i,j in data.ponds_nodes_data['avg_area'].items():
    print(i)
    total_evap = data.evaporation_rates * 6.28981077 * data.ponds_nodes_data['avg_area'][i]
    print("sum", total_evap.evaporation_rate.sum())
    print("avg", total_evap.evaporation_rate.mean())

    TOTAL_EVAP +=total_evap.evaporation_rate.sum()
print('daily avg',TOTAL_EVAP/365)

POND_22_1
134636.63650043597
368.86749726146843
POND_26_1
114541.5149030759
313.81236959746826
POND_27_1
127291.71017915993
348.7444114497532
POND_27_2
193721.58397131035
530.7440656748229
OXY_BRACKISH_PONDS_SOUTH_10_1
99919.48244816254
273.75200670729464
3P_MASK_POND
328419.2586406212
899.7787907962224
RECYCLING_POND_1
540282.7428052956
1480.2266926172483
RECYCLING_POND_2
540282.7428052956
1480.2266926172483
RECYCLING_POND_3
540282.7428052956
1480.2266926172483
RECYCLING_POND_4
331125.28412740707
907.1925592531701
RECYCLING_POND_5
368887.56460466015
1010.6508619305757
daily avg 9094.222640522521


In [30]:
for i,j in useful_sets.exits.items():
    if i in useful_sets.pond_nodes:
        # useful_sets.loss_tank_nodes
        print(i)
        print(j)

POND_22_1
{'BRACKISH_POND_22_1_SPLITTER', 'LOSS_22_1'}
RECYCLING_POND_5
{'LOSS_RECYC_POND_5', 'RECYCLING_POND_5_SPLIT'}
POND_26_1
{'LOSS_26_1', 'BRACKISH_POND_26_1_SPLITTER'}
3P_MASK_POND
{'LOSS_MASK_POND', 'POND_3P_SPLIT'}
POND_27_2
{'LOSS_27_2', 'BRACKISH_POND_27_2_SPLITTER'}
POND_27_1
{'LOSS_27_1', 'BRACKISH_POND_27_1_SPLITTER'}
RECYCLING_POND_2
{'LOSS_RECYC_POND_2', 'RECYCLING_POND_2_SPLIT'}
OXY_BRACKISH_PONDS_SOUTH_10_1
{'LOSS_10_1', 'OXY_BRACKISH_PONDS_SOUTH_SPLIT'}
RECYCLING_POND_3
{'LOSS_RECYC_POND_3', 'RECYCLING_POND_3_SPLIT'}
RECYCLING_POND_4
{'RECYCLING_SOUTH_SPLIT', 'LOSS_RECYC_POND_4'}
RECYCLING_POND_1
{'LOSS_RECYC_POND_1', 'RECYCLING_POND_1_SPLIT'}


In [9]:
model, solver, result = make_model(data, useful_sets, parameters,attributes_with_time)

        [2023-04-06 10:52:58.618931] Creating ConcreteModel...
        [2023-04-06 10:52:58.618931] Creating sets...
        [2023-04-06 10:52:58.621317] Creating parameters...
        [2023-04-06 10:52:58.638522] Creating Variables...
        [2023-04-06 10:52:58.640517] Creating Constraints...
        [2023-04-06 10:52:58.651229] Creating Objectives...
      Solving the objective function number 1
        0.01 seconds required to write file
        0.01 seconds required for presolve
        7.23 seconds required for solver
        0.00 seconds required to read logfile 
        0.01 seconds required to read solution file 
        0.02 seconds required for postsolve
        Solver status: ok
        Solver termination condition: optimal
Solution count: 1
        Optimal value: 0.0
        New constraint: calculate_delta_water_nominal 1: should be less than or equal to 0.0
      Solving the objective function number 2
        0.01 seconds required to write file
Warm start write time=0.0

Sequential Run of optimize.py

In [2]:
s3_data, param_file = False, "parameters_AWS.json"  #please maintain this values: False and None, once you have finished your development

In [3]:
date = None if re.match('linux.*', sys.platform) else datetime.datetime.now()
system_utilities = SystemUtilities(s3_data, param_file=param_file, date=date)
system_utilities.read_parameters()
system_utilities.generate_parameters('water')

    reading parameters file parameters_AWS.json...
    processing for treatment asset...
    reading parameters...


In [4]:
parameters = system_utilities.parameters
#defining objects to store outputs
athena, times, pandas_dataframe = [], {}, None
delta, iterations = parameters['delta'], parameters['iterations']


In [5]:
model, result, runtime = treatment_model('water', s3_data, param_file, date, None)


**************************************************
RUNNING WATER MODEL:
    reading parameters...
    reading parameters file parameters_AWS.json...
    processing for treatment asset...
    checking directories...
    reading parameters...
    Preprocessing data
    reading data from one drive...
    processing loaded data...
    creating class attributes...
    optimizing model...
        [2023-04-06 12:39:32.411172] Creating ConcreteModel...
        [2023-04-06 12:39:32.411712] Creating sets...
        [2023-04-06 12:39:32.419797] Creating parameters...
        [2023-04-06 12:39:35.577344] Creating Variables...
        [2023-04-06 12:39:35.582965] Creating Constraints...
        [2023-04-06 12:39:35.611081] Creating Objectives...
      Solving the objective function number 1
        0.05 seconds required to write file
        0.05 seconds required for presolve
        7.74 seconds required for solver
        0.00 seconds required to read logfile 
        0.02 seconds required to rea

In [12]:
model_athena = process_treatment_results('water', s3_data, param_file, date, None, model, result)


    reading parameters file parameters_AWS.json...
    processing for treatment asset...
    checking directories...
    reading parameters...
    reading data from one drive...
    processing loaded data...
    creating class attributes...
    exporting results...
PROCESSING WATER MODEL OUTPUTS:
    reading parameters file parameters_AWS.json...
    processing for treatment asset...
    reading parameters...
    reading data from one drive...
    processing edges...
    processing nodes...
    putting it together...
    addign kpis to database...
    WATER running done.
**************************************************


In [6]:
#Running postprocess sequentially
model_name = 'water'
cost_of_injection = None
test, parameters, simulated_period = read_parameters(model_name, s3_data, param_file, date)
data, _, _ = preprocess_data(parameters, simulated_period,s3_data=s3_data, cost_of_injection = cost_of_injection)


    reading parameters file parameters_AWS.json...
    processing for treatment asset...
    checking directories...
    reading parameters...
    reading data from one drive...
    processing loaded data...
    creating class attributes...


In [7]:
outputs = generate_output(model, result, parameters, data)


In [8]:
#generating recirculation recomendations
recirculation = wr.get_action_recirculation(data.arcs_data.reset_index(), outputs)
#generating reuse recomendations
reuse = wr.get_action_reuse(data.arcs_data.reset_index(), data.ending_nodes_data.reset_index(), outputs)
#generating nominal value recomendations
nominal = wr.get_action_nominal_values(data.arcs_data.reset_index(), outputs)
recommendations, actions = [recirculation, reuse, nominal], []
for recomm in recommendations:
    print(recomm)
    if len(recomm):
        actions.extend(recomm)
if len(actions):
    recomendations_path = test.parameters["output_recommendations_dir"]
    with open(recomendations_path, 'w') as f:
        json.dump(actions, f)


[{'time': 1, 'type': 'recirculate_per_arc', 'params': {'value': 9999899.0, 'percentage': 3999.9596, 'source': 'G', 'target': 'I'}}, {'time': 1, 'type': 'recirculate_system', 'params': {'value': 9999899.0, 'percentage': 3999.9596, 'source': 'model_start', 'target': 'model_end'}}]
[]
[{'time': 1, 'type': 'nominal_value_recommend', 'params': {'percentage fulfillment of nominal': 100.0, 'source': 'F', 'target': 'I', 'Minimum operative window': 0, 'Maximum operative window': 9999999, 'optimal water': 100.0, 'desired water': 100.0}}]


In [11]:
print(f'PROCESSING {model_name.upper()} MODEL OUTPUTS:')
test_results = ProcessResults(model_name, outputs,  s3_data=s3_data, param_file=param_file, date=date)
print(f"    {model_name.upper()} running done.")
print('*'*50)



PROCESSING WATER MODEL OUTPUTS:
    reading parameters file parameters_AWS.json...
    processing for treatment asset...
    reading parameters...
    reading data from one drive...
    processing edges...
    processing nodes...
    putting it together...
    addign kpis to database...
    WATER running done.
**************************************************


In [ ]:
athena, times = save_optimization_results(system_utilities, athena, times, 'water', model, model_athena, runtime)


In [56]:
#EVAPORATION
import pandas as pd
import numpy as np
from datetime import datetime
from scipy import optimize
from scipy.optimize import root_scalar

# Set the target evaporation and initial guess for the correction factor
target_evap_barrels = 8500 #barrels/day (for the whole surface of the facility)
target_evap_m3 = target_evap_barrels * 0.1589873
tot_surface_area = 217524 #unit = m2

data_folder = 'C:/Users/david.collard/Accenture/HTX Hub - Ecopetrol Water Management - 07. Rodeo Midland Basin - Work. Sol/300_Create/Analytics/Analysis/Weather Forecasting/Data sources/'
weather_data = pd.read_excel(data_folder + 'Open weather API Permian structured.xlsx', sheet_name = 'historic data')
radiation_data = pd.read_excel(data_folder + 'SoDa_Data_Permian_lat32.209_lon-102.134_2005-01-01_2022-12-31.xlsx', sheet_name = 'historic data')


In [57]:
def preprocessing_weather(weather_data):
        # create separate columns that indicate the month and the year
    weather_data['date_short'] = [x[:10] for x in weather_data['dt_iso']]
    weather_data["month"] = [x[5:7] for x in weather_data['date_short']]
    weather_data["year"] = [x[0:4] for x in weather_data['date_short']]
    weather_data['temp_celcius'] = weather_data['temp'] - 273.15

    # limit the weather data to the dates we have available for solar data - 1/1/2005 until 31/12/2022
    weather_data["year"] = weather_data["year"].astype('int')
    weather_data_limited = weather_data[weather_data["year"] >= 2005].reset_index(drop=True)
    weather_data_limited = weather_data_limited[weather_data_limited["year"] <= 2022].reset_index(drop=True)
    return weather_data_limited

def preprocessing_radiation(radiation_data):
    #change dates to same format as in weather data by adding a zero before single numbers
    # the method is called leading zero's and I want 2 numbers in total so one leading zero
    radiation_data['Month']=radiation_data['Month'].apply(lambda x: '{0:0>2}'.format(x))
    radiation_data['Day']=radiation_data['Day'].apply(lambda x: '{0:0>2}'.format(x))
    radiation_data['date_short'] = radiation_data["Year"].astype(str) + "-" + radiation_data["Month"] + "-" + radiation_data["Day"]
    # rename columns to the same format as weather data
    radiation_data = radiation_data.rename(columns={"Year": "year", "Month": "month", "Extraterrestrial Irradiance": "extraterrestial_irradiance", "Irradiation (Wh/m2)":"solar_irradiance"})
    # select only the columns we want
    radiation_data = radiation_data[["date_short", "month", "year", "extraterrestial_irradiance", "solar_irradiance"]]
    return radiation_data

def merge_weather_radiation(weather_data, radiation_data):
    weather_radiation_data = weather_data.merge(radiation_data, left_on='date_short', right_on='date_short', suffixes=('_weather', '_radiation'))
    # select only the columns we want
    weather_radiation_data = weather_radiation_data[["date_short", "month_weather", "year_weather", "temp_celcius", "humidity", "wind_speed", "extraterrestial_irradiance", "solar_irradiance"]]
    # rename columns
    weather_radiation_data = weather_radiation_data.rename(columns={"date_short": "Date", "month_weather": "month", "year_weather": "year"})
    # calculate irradiance from Wh/m2 to MJ/m2 by using 1 MJ/m2 = 0.0036 * Wh/m2
    weather_radiation_data["extraterrestial_irradiance_MJ"] = weather_radiation_data["extraterrestial_irradiance"]*0.0864
    weather_radiation_data["solar_irradiance_MJ"] = (weather_radiation_data["solar_irradiance"]*0.0036)
    return weather_radiation_data

#Preparing the weather_radiation dataframe
weather_data = preprocessing_weather(weather_data)
radiation_data = preprocessing_radiation(radiation_data)
weather_radiation_data = merge_weather_radiation(weather_data, radiation_data)


In [58]:
def calculate_evaporation(correction_factor): #weather_radiation_data, target_evap, 

    # Apply correction factor 
    weather_radiation_data["solar_irradiance_MJ_NEW"] = weather_radiation_data["solar_irradiance_MJ"] * correction_factor

    # Parts of the evaporation calculation 
    weather_radiation_data["EQ33_formula_part1"] = 0.051 * (1-0.08) * weather_radiation_data['solar_irradiance_MJ_NEW'] * (((weather_radiation_data['temp_celcius'])+9.5)**(1/2))
    weather_radiation_data["EQ33_formula_part2"] = 2.4 * ((weather_radiation_data['solar_irradiance_MJ_NEW']/weather_radiation_data['extraterrestial_irradiance_MJ'])**(2))
    weather_radiation_data["EQ33_formula_part3"] = 0.09 * (weather_radiation_data['temp_celcius']+20) * (1-(weather_radiation_data['humidity']/100))

    # Bring all parts of the equation together
    weather_radiation_data["Evaporation"] = weather_radiation_data["EQ33_formula_part1"] - weather_radiation_data["EQ33_formula_part2"] + weather_radiation_data["EQ33_formula_part3"]

    # Convert evaporation in mm/day to m/day to be in line with other units in the water management model
    weather_radiation_data["Evaporation"] = weather_radiation_data['Evaporation'] * 0.001

    # Get evaporation lost
    weather_radiation_data["Evaporation_Loss"] = weather_radiation_data["Evaporation"] * tot_surface_area

    # Get only the columns we need
    calculated_evaporation = weather_radiation_data[["Date", "year", "Evaporation_Loss"]] 

    # Get current year to filter the mean evaporation so that we can compare it with target evaporation
    evaporation_average = calculated_evaporation.groupby("year").agg(evaporation_loss = ('Evaporation_Loss',"mean")).reset_index()
    # Filter on last year
    last_year = datetime.now().year - 1
    evaporation_average = evaporation_average.evaporation_loss[evaporation_average.year==last_year]

    print("evaporation avg:", float(evaporation_average))
    print("target:", target_evap_m3)

    evap_difference = evaporation_average - target_evap_m3

    return float(evap_difference)

In [61]:
diff = calculate_evaporation(correction_factor = 0.7) #weather_radiation_data = weather_radiation_data, target_evap = target_evap_m3, 
print(diff)


evaporation avg: 1608.9791461891637
target: 1351.39205
257.58709618916373


In [63]:
result = root_scalar(calculate_evaporation, bracket= [0,1]) #args=(weather_radiation_data, target_evap_m3)

# The correction factor will be stored in the 'root' attribute of the result object
new_correction_factor = result.root
print(new_correction_factor)

# Use the correction factor to adjust your evaporation calculation
adjusted_evap = calculate_evaporation(correction_factor = new_correction_factor) #weather_radiation_data = weather_radiation_data, target_evap = target_evap_m3,
print(adjusted_evap)

evaporation avg: 425.69694954930634
target: 1351.39205
evaporation avg: 2104.223759654534
target: 1351.39205
evaporation avg: 1361.1840235298973
target: 1351.39205
evaporation avg: 1351.3893451123247
target: 1351.39205
evaporation avg: 1351.392050373866
target: 1351.39205
evaporation avg: 1351.39205
target: 1351.39205
0.5456455247698183
evaporation avg: 1351.39205
target: 1351.39205
0.0
